In [1]:
from skillcorner.client import SkillcornerClient

import pandas as pd

import numpy as np

import os

secret_password = os.getenv("mdp_skillcorner")
client = SkillcornerClient(username = "Nathan.talbot@etu.uca.fr", password = secret_password)

In [29]:
dico_annee = {
    "2021/2022" : 243,
    "2022/2023" : 393,
    "2023/2024" : 549
}

liste_met = pd.read_excel("../../Tableau métriques/liste_métriques.xlsx", header = 0)
liste_met = liste_met["passes"].dropna()

df = pd.DataFrame(index = liste_met, columns = dico_annee.keys())

for i in dico_annee.keys() :
    data_import = client.get_in_possession_passes(params = {'competition_edition': dico_annee[i],
          "run_type" : ["run_in_behind", "run_ahead_of_the_ball", "support_run", "pulling_wide_run", "coming_short_run", "underlap_run",
                    "overlap_run", "dropping_off_run", "pulling_half_space_run", "cross_receiver_run"]})
    data = pd.DataFrame(data_import).set_index("team_name")
    data = data[data.quality_check == True]
    data.fillna(0, inplace = True)
    nb_matchs = pd.Series(index = data.index.unique())
    for team in nb_matchs.index :
        nb_matchs[team] = len(data.loc[team].match_id.unique())

    drop = ["quality_check", "player_id", "player_name", "short_name", "player_birthdate", "match_id", "match_name", "match_date", "team_id",
        "competition_id", "competition_name", "season_id", "season_name", "competition_edition_id", "position", "group", "result", "venue",
        "third", "channel", "minutes_played_per_match", "adjusted_min_tip_per_match"]
    data.drop(drop, inplace = True, axis = 1)

    data = data.groupby("team_name").sum()

    data = data.divide(nb_matchs, axis = 0)
    df[i] = data.mean(axis = 0)

    df["Évolution en %"] = 100*(df["2023/2024"] - df["2021/2022"])/df["2021/2022"]

    df.sort_values(by = "Évolution en %", ascending = False, inplace = True)

In [35]:
df

,2021/2022,2022/2023,2023/2024,Évolution en %
passes,,,,
count_completed_pass_to_pulling_wide_runs_leading_to_goal_per_match,0.008238,0.017144,0.025436,208.757884
count_completed_pass_to_underlap_runs_leading_to_goal_per_match,0.012577,0.015421,0.037715,199.865063
count_pass_attempts_to_dangerous_dropping_off_runs_per_match,0.005648,0.007271,0.012095,114.131204
count_completed_pass_to_dropping_off_runs_leading_to_shot_per_match,0.070984,0.108379,0.125552,76.873289
count_pass_attempts_to_dangerous_pulling_wide_runs_per_match,0.014530,0.005875,0.025251,73.784880
...,...,...,...,...
count_pass_attempts_to_pulling_half_space_runs_per_match,2.474640,2.347977,2.393745,-3.268970
count_pass_opportunities_to_dangerous_coming_short_runs_per_match,0.111150,0.106628,0.106721,-3.984918
count_completed_pass_to_dropping_off_runs_leading_to_goal_per_match,0.005522,0.002899,0.005269,-4.588037
